In [1]:
import os
import random
import sys
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

sys.path.append("..")
import warnings

import wandb
from otc.metrics.metrics import effective_spread
from scipy.stats import wilcoxon
from tqdm.auto import tqdm


/home/kit/stud/uloak/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# set here globally
EXCHANGE = "cboe" # "ise"
MODELS = ["fttransformer", "classical"] 
SUBSET = "test"  # "all"
STRATEGY = "transfer" # "supervised" #  


In [3]:
KEY = f"{EXCHANGE}_{STRATEGY}_{SUBSET}"
DATASET = f"fbv/thesis/{EXCHANGE}_{STRATEGY}_raw:latest"

os.environ["GCLOUD_PROJECT"] = "flowing-mantis-239216"

run = wandb.init(project="thesis", entity="fbv")

# load unscaled data
artifact = run.use_artifact(DATASET)  # type: ignore
data_dir = artifact.download()

# load results
result_dirs = []
for model in MODELS:
    results = f"fbv/thesis/{EXCHANGE}_{model}_{STRATEGY}_{SUBSET}:latest"
    artifact = run.use_artifact(results)  # type: ignore
    result_dir = artifact.download()
    result_dirs.append(result_dir)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: karelze (fbv). Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact ise_supervised_raw:latest, 2589.45MB. 3 files... 
wandb:   3 of 3 files downloaded.  
Done. 0:0:3.4
wandb: Downloading large artifact ise_fttransformer_supervised_test:latest, 72.26MB. 1 files... 
/pfs/data5/home/kit/stud/uloak/thesis/standard/lib64/python3.9/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
wandb:   1 of 1 files downloaded.  
Done. 0:0:1.8
wandb: Downloading large artifact ise_classical_supervised_test:latest, 499.12MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:1.1


In [4]:
# p. 35-38
COLUMNS = [
    "buy_sell",
    "EXPIRATION",
    "QUOTE_DATETIME",
    "TRADE_SIZE",
    "TRADE_PRICE",
    "ask_ex",
    "bid_ex",
    "myn",
    "OPTION_TYPE",
    "issue_type",
]


if SUBSET == "all":
    train = pd.read_parquet(
        Path(data_dir, "train_set"), engine="fastparquet", columns=COLUMNS
    )
    val = pd.read_parquet(
        Path(data_dir, "val_set"), engine="fastparquet", columns=COLUMNS
    )
    test = pd.read_parquet(
        Path(data_dir, "test_set"), engine="fastparquet", columns=COLUMNS
    )
    eval_data = pd.concat([train, val, test])
    del train, val, test

elif SUBSET == "test":
    eval_data = pd.read_parquet(
        Path(data_dir, "test_set"), engine="fastparquet", columns=COLUMNS
    )


results = []
for i, model in tqdm(enumerate(MODELS)):
    result = pd.read_parquet(Path(result_dirs[i], "results"), engine="fastparquet")
    result.columns = pd.MultiIndex.from_product([[model], result.columns])
    results.append(result)

results_data = pd.concat(results, axis=1, names=MODELS)

assert len(eval_data) == len(results_data)

X_print = eval_data

del results


2it [00:04,  2.06s/it]


In [5]:
# FIXME: select a subset of results for testing.
results_data = results_data[
    [
        ("fttransformer", "fttransformer(classical)"),
        ("fttransformer", "fttransformer(classical-size)"),
        ("fttransformer", "fttransformer(ml)"),        
        #("gbm", "gbm(classical)"),
        # ("gbm", "gbm(classical-size)"),
        #("gbm", "gbm(ml)"),
        # ("gbm", "gbm(classical-retraining)"),
        # ("gbm", "gbm(classical-size-retraining)"),
        # ("gbm", "gbm(ml-retraining)"),
        # ("gbm", "gbm(semi-classical)"),
        # ("gbm",'gbm(semi-classical-retraining)'),
        ("classical", "tick(ex)"),
        ("classical", "quote(ex)"),
        ("classical", "lr(ex)"),
        ("classical", "emo(ex)"),
        ("classical", "clnv(ex)"),
        ("classical", "quote(best)->quote(ex)"),
        (
            "classical",
            "trade_size(ex)->quote(best)->quote(ex)->depth(best)->depth(ex)->rev_tick(all)",
        ),
    ]
]


In [6]:
results_data


fttransformer                                \
         fttransformer(classical) fttransformer(classical-size)   
index                                                             
39342191                       -1                            -1   
39342190                       -1                            -1   
39342189                        1                             1   
39342188                        1                            -1   
39342187                       -1                            -1   
...                           ...                           ...   
49203742                       -1                            -1   
49203743                        1                             1   
49203744                        1                             1   
49203745                        1                             1   
49203746                        1                             1   

                           classical                                    \
         fttransformer(ml)  tick(ex) quote(ex) lr(ex) emo(ex) clnv(ex)   
index                                                                    
39342191                -1         1        -1     -1      -1       -1   
39342190                -1        -1        -1     -1      -1       -1   
39342189                 1         1         1      1       1        1   
39342188                -1        -1        -1     -1      -1       -1   
39342187                -1         1        -1     -1      -1       -1   
...                    ...       ...       ...    ...     ...      ...   
49203742                -1        -1         1      1       1        1   
49203743                 1        -1         1      1      -1        1   
49203744                -1        -1        -1     -1      -1       -1   
49203745                 1        -1         1      1       1        1   
49203746                 1         1         1      1       1        1   

                                 \
         quote(best)->quote(ex)   
index                             
39342191                     -1   
39342190                     -1   
39342189                      1   
39342188                     -1   
39342187                     -1   
...                         ...   
49203742                      1   
49203743                      1   
49203744                     -1   
49203745                      1   
49203746                      1   

                                                                                        
         trade_size(ex)->quote(best)->quote(ex)->depth(best)->depth(ex)->rev_tick(all)  
index                                                                                   
39342191                                                 -1                             
39342190                                                 -1                             
39342189                                                  1                             
39342188                                                 -1                             
39342187                                                 -1                             
...                                                     ...                             
49203742                                                  1                             
49203743                                                  1                             
49203744                                                 -1                             
49203745                                                  1                             
49203746                                                  1                             

[9861576 rows x 10 columns]

In [7]:
LUT = {
    "Trade_Size(ex)->Quote(Best)->Depth(Best)->Quote(Ex)->Depth(Ex)->Rev_Tick(All)": "\gls{GBM}",
    "(Ex)": " (Ex)",
    "(Best)": " (Best)",
    "(Classical)": " (Classical)",
    "(Classical-Size)": " (Classical, Size)",
    "Rev_": "Rev. ",
    "Trade_Size": "Trade Size",
    "Depth": "Depth",
    "->": " $\\to$ ",
    "Lr": "\gls{LR}",
    "Emo": "\gls{EMO}",
    "Clnv": "\gls{CLNV}",
    "OPTION_TYPE": "Option Type",
    "_": "$\_",
    "Gbm": "\gls{GBM}",
}

LUT_INDEX = {
    "OPTION_TYPE": "Option Type",
    "issue_type": "Security Type",
    "TRADE_SIZE_binned": "Trade Size",
    "year_binned": "Year",
    "ttm_binned": "Time to Maturity",
    "myn_binned": "Moneyness",
    "prox_q_binned": "Location to Quote",
    "all": "All trades",
}


def cell_str(x):
    x = x.title()
    for orig, sub in LUT.items():
        x = x.replace(orig, sub)
    # title-case everything
    return x


def highlight_max(s, props=""):
    return np.where(s == np.nanmax(s.values), props, "")


In [8]:
def set_tex_style(styler, caption, label, bold_axis=1):
    res = styler.set_caption(caption)

    res = (
        res.apply(highlight_max, props="font-weight:bold;", axis=bold_axis)
        .format(precision=4, decimal=".", thousands=",", escape=False, hyperlinks=None)
        .format_index(cell_str, axis=0)
        .format_index(cell_str, axis=1)
        .to_latex(
            f"{label}.tex",
            siunitx=True,
            position_float="centering",
            hrules=True,
            clines="skip-last;data",
            label="tab:" + label,
            caption=caption,
            convert_css=True,
        )
    )
    return res


In [9]:
classifiers = results_data.columns.tolist()
criterions = list(LUT_INDEX)


## Unclassified by method

In [10]:
results_data.head()


fttransformer                                \
         fttransformer(classical) fttransformer(classical-size)   
index                                                             
39342191                       -1                            -1   
39342190                       -1                            -1   
39342189                        1                             1   
39342188                        1                            -1   
39342187                       -1                            -1   

                           classical                                    \
         fttransformer(ml)  tick(ex) quote(ex) lr(ex) emo(ex) clnv(ex)   
index                                                                    
39342191                -1         1        -1     -1      -1       -1   
39342190                -1        -1        -1     -1      -1       -1   
39342189                 1         1         1      1       1        1   
39342188                -1        -1        -1     -1      -1       -1   
39342187                -1         1        -1     -1      -1       -1   

                                 \
         quote(best)->quote(ex)   
index                             
39342191                     -1   
39342190                     -1   
39342189                      1   
39342188                     -1   
39342187                     -1   

                                                                                        
         trade_size(ex)->quote(best)->quote(ex)->depth(best)->depth(ex)->rev_tick(all)  
index                                                                                   
39342191                                                 -1                             
39342190                                                 -1                             
39342189                                                  1                             
39342188                                                 -1                             
39342187                                                 -1

In [11]:
unclassified = (
    (results_data[results_data == 0.0].count(axis=0) / len(results_data.index))
    .sort_values(ascending=False)
    .to_frame(name="unclassified")
)


In [12]:
unclassified.style.pipe(
    set_tex_style,
    caption=(f"{KEY}-unclassified-long", "{key}-unclassified-short"),
    label=f"{KEY.lower()}-unclassfied",
    bold_axis=0,
)
unclassified


unclassified
classical     tick(ex)                                                0.124762
              quote(ex)                                               0.060195
              quote(best)->quote(ex)                                  0.037738
              emo(ex)                                                 0.023059
              clnv(ex)                                                0.019444
              lr(ex)                                                  0.001618
              trade_size(ex)->quote(best)->quote(ex)->depth(b...      0.000043
fttransformer fttransformer(classical)                                0.000000
              fttransformer(classical-size)                           0.000000
              fttransformer(ml)                                       0.000000

## Fill in unclassified

Unclassified are `0`.

In [13]:
rng = np.random.default_rng(42)

# replace 0 -> nan -> [-1,1]
results_data.replace(0, np.nan, inplace=True)
filler = pd.DataFrame(
    rng.choice(a=[-1, 1], size=results_data.shape),
    index=results_data.index,
    columns=results_data.columns,
)
results_data.fillna(filler, inplace=True)


### Robustness Checks

In [14]:
# prepare columns for printing
X_print["ttm"] = (
    X_print["EXPIRATION"].dt.to_period("M")
    - X_print["QUOTE_DATETIME"].dt.to_period("M")
).apply(lambda x: x.n)

X_print["year"] = X_print["QUOTE_DATETIME"].dt.year

bins_tradesize = [-1, 1, 3, 5, 11, np.inf]
trade_size_labels = ["(0,1]", "(1,3]", "(3,5]", "(5,11]", ">11"]
X_print["TRADE_SIZE_binned"] = pd.cut(
    X_print["TRADE_SIZE"], bins_tradesize, labels=trade_size_labels
)

# p. 38
bins_years = [2004, 2007, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]
year_labels = [
    "2005-2007",
    "2008-2010",
    "2011",
    "2012",
    "2013",
    "2014",
    "2015",
    "2016",
    "2017",
]
X_print["year_binned"] = pd.cut(X_print["year"], bins_years, labels=year_labels)

# p. 37
bins_ttm = [-1, 1, 2, 3, 6, 12, np.inf]
ttm_labels = [
    "<= 1",
    "(1-2]",
    "(2-3]",
    "(3-6]",
    "(6-12]",
    "> 12",
]
X_print["ttm_binned"] = pd.cut(X_print["ttm"], bins_ttm, labels=ttm_labels)

# Security type
# see 3.0a-mb-explanatory-data-analysis.ipynb
X_print["issue_type"] = X_print["issue_type"].map(
    {
        "0": "Stock option",
        "A": "Index option",
        "7": "Others",
        "F": "Others",
        "%": "Others",
        " ": "Others",
    }
)

# Moneyness p. 38
bins_myn = [-1, 0.7, 0.9, 1.1, 1.3, np.inf]
myn_labels = [
    "<= 0.7",
    "(0.7-0.9]",
    "(0.9-1.1]",
    "(1.1-1.3]",
    "> 1.3",
]
X_print["myn_binned"] = pd.cut(X_print["myn"], bins_myn, labels=myn_labels)

# mid p. 31 + extra category for unknowns
ask = X_print["ask_ex"]
bid = X_print["bid_ex"]
trade_price = X_print["TRADE_PRICE"]

# require ask >= bid
mid = np.where(ask >= bid, (ask + bid) * 0.5, np.nan)

prox_quotes = np.where(
    trade_price == mid,
    0,  # at mid
    np.where(
        (bid < trade_price) & (trade_price < ask),
        1,  # inside
        np.where(
            (trade_price == bid) | (ask == trade_price),
            2,  # at quotes
            np.where((trade_price < bid) | (ask < trade_price), 3, 4),
        ),
    ),
)  # outside + unclassifiable

bins_prox = [-np.inf, 0, 1, 2, 3, 4]
prox_labels = [
    "at mid",
    "inside",
    "at quotes",
    "outside",
    "unknown",
]

X_print["prox_q_binned"] = pd.cut(prox_quotes, bins_prox, labels=prox_labels)
X_print["mid"] = mid

# clean up empty buckets, as it causes empty grouping in result set generation
X_print["year_binned"] = X_print["year_binned"].cat.remove_unused_categories()
X_print["myn_binned"] = X_print["myn_binned"].cat.remove_unused_categories()
X_print["ttm_binned"] = X_print["ttm_binned"].cat.remove_unused_categories()
X_print["prox_q_binned"] = X_print["prox_q_binned"].cat.remove_unused_categories()

X_print["all"] = "all"

X_print.drop(
    columns=[
        "EXPIRATION",
        "QUOTE_DATETIME",
        "TRADE_SIZE",
        "ttm",
        "myn",
        "year",
    ],
    inplace=True,
)


In [15]:
X_print = pd.concat([X_print, results_data], axis=1)


In [16]:
X_print.head().T


index,39342191,39342190,39342189,39342188,39342187
buy_sell,-1,-1,1,-1,-1
TRADE_PRICE,3.5,6.38,0.13,0.04,0.4
ask_ex,3.85,6.8,0.13,0.1,0.55
bid_ex,3.5,6.35,0.07,0.04,0.31
OPTION_TYPE,P,C,P,C,C
issue_type,Stock option,Stock option,Others,Others,Others
TRADE_SIZE_binned,"(3,5]","(0,1]","(0,1]","(1,3]","(3,5]"
year_binned,2015,2015,2015,2015,2015
ttm_binned,<= 1,<= 1,<= 1,<= 1,<= 1
myn_binned,(0.9-1.1],(0.9-1.1],(0.7-0.9],(0.7-0.9],(0.9-1.1]


## Accuracy Calculation

In [17]:
# FIXME: Find better approach
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

result_dfs = []

for criterion in tqdm(criterions):
    results = []
    for classifier in tqdm(classifiers):
        res = (
            X_print.groupby([criterion])[["buy_sell", classifier]]
            .apply(
                lambda x: accuracy_score(x["buy_sell"].astype("int8"), x[classifier])
            )
            .mul(100)
            .rename(classifier)
        )
        #         acc_tot = accuracy_score(
        #             X_print["buy_sell"].astype("int8"), X_print[classifier]
        #         )

        #         res.loc["all"] = acc_tot * 100

        res.index.name = LUT_INDEX.get(criterion)
        results.append(res)

    # save aggregated results
    result_df = pd.concat(results, axis=1).T
    result_df.style.pipe(
        set_tex_style,
        caption=(f"long-tbd", "short-tbd"),
        label=f"{KEY.lower()}-{criterion.lower()}",
    )

    # store all result sets for later use
    result_dfs.append(result_df)


100%|██████████| 8/8 [01:55<00:00, 14.42s/it]


In [18]:
master = pd.concat(result_dfs, axis=1, keys=list(LUT_INDEX.values())).T


In [19]:
master


fttransformer  \
                               fttransformer(classical)   
Option Type       C                           61.552726   
                  P                           62.942856   
Security Type     Index option                53.741246   
                  Others                      67.140345   
                  Stock option                60.345998   
Trade Size        (0,1]                       60.574816   
                  (1,3]                       61.065922   
                  (3,5]                       61.223988   
                  (5,11]                      64.294364   
                  >11                         65.643297   
Year              2015                        59.149311   
                  2016                        62.405291   
                  2017                        62.757030   
Time to Maturity  <= 1                        62.822214   
                  (1-2]                       63.277653   
                  (2-3]                       61.796898   
                  (3-6]                       60.605037   
                  (6-12]                      60.658303   
                  > 12                        54.980419   
Moneyness         <= 0.7                      64.418661   
                  (0.7-0.9]                   65.849035   
                  (0.9-1.1]                   62.596430   
                  (1.1-1.3]                   53.180114   
                  > 1.3                       51.737642   
Location to Quote at mid                      61.401592   
                  inside                      60.753892   
                  at quotes                   67.592344   
                  outside                     60.041602   
                  unknown                     77.824153   
All trades        all                         62.201670   

                                                              \
                               fttransformer(classical-size)   
Option Type       C                                70.755864   
                  P                                71.672920   
Security Type     Index option                     56.597390   
                  Others                           75.306743   
                  Stock option                     69.743643   
Trade Size        (0,1]                            71.734785   
                  (1,3]                            72.174763   
                  (3,5]                            71.912851   
                  (5,11]                           70.025028   
                  >11                              69.348202   
Year              2015                             68.112642   
                  2016                             71.430327   
                  2017                             71.646482   
Time to Maturity  <= 1                             71.513974   
                  (1-2]                            71.695599   
                  (2-3]                            70.753307   
                  (3-6]                            70.267933   
                  (6-12]                           70.127499   
                  > 12                             68.230785   
Moneyness         <= 0.7                           70.345507   
                  (0.7-0.9]                        72.879601   
                  (0.9-1.1]                        71.774622   
                  (1.1-1.3]                        65.846414   
                  > 1.3                            62.633377   
Location to Quote at mid                           71.010869   
                  inside                           67.336010   
                  at quotes                        84.802591   
                  outside                          63.329531   
                  unknown                          77.297297   
All trades        all                              71.183967   

                                                  classical             \
                              

In [20]:
master.style.pipe(
    set_tex_style,
    caption=("master-long", "master-short"),
    label=f"{KEY}-master",
    bold_axis=0,
)


## Effective Spread 💴

In [21]:
X_print.head()

,buy_sell,TRADE_PRICE,ask_ex,bid_ex,OPTION_TYPE,issue_type,TRADE_SIZE_binned,year_binned,ttm_binned,myn_binned,...,"(gbm, gbm(classical))","(gbm, gbm(classical-size))","(gbm, gbm(ml))","(classical, tick(ex))","(classical, quote(ex))","(classical, lr(ex))","(classical, emo(ex))","(classical, clnv(ex))","(classical, quote(best)->quote(ex))","(classical, trade_size(ex)->quote(best)->quote(ex)->depth(best)->depth(ex)->rev_tick(all))"
index,,,,,,,,,,,,,,,,,,,,,
39342191,-1,3.50,3.85,3.50,P,Stock option,"(3,5]",2015,<= 1,(0.9-1.1],...,-1,-1,-1,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
39342190,-1,6.38,6.80,6.35,C,Stock option,"(0,1]",2015,<= 1,(0.9-1.1],...,-1,-1,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
39342189,1,0.13,0.13,0.07,P,Others,"(0,1]",2015,<= 1,(0.7-0.9],...,1,1,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0
39342188,-1,0.04,0.10,0.04,C,Others,"(1,3]",2015,<= 1,(0.7-0.9],...,-1,-1,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
39342187,-1,0.40,0.55,0.31,C,Others,"(3,5]",2015,<= 1,(0.9-1.1],...,-1,-1,-1,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [22]:
results = []

ask = X_print["ask_ex"]
bid = X_print["bid_ex"]
mid = X_print["mid"]

# calculate true rel effective spread but not aggregated, convert to %
es_true = effective_spread(X_print["buy_sell"], X_print["TRADE_PRICE"], mid, mode="none")
eps_true = np.empty(es_true.shape)
np.divide(es_true, mid, out=eps_true, where=mid != 0)

nom_true = effective_spread(X_print["buy_sell"], X_print["TRADE_PRICE"], mid, mode="nominal")
rel_true = effective_spread(X_print["buy_sell"], X_print["TRADE_PRICE"], mid, mode="relative")

# require ask > bid
rel_quoted = np.nanmean((ask - bid) / mid)
nom_quoted = np.nanmean(np.where(ask >= bid, (ask - bid), np.nan))

for classifier in tqdm(classifiers):
    nom_pred = effective_spread(X_print[classifier], X_print["TRADE_PRICE"], mid, mode="nominal")
    rel_pred = effective_spread(X_print[classifier], X_print["TRADE_PRICE"], mid, mode="relative")

    # calculate pred rel effective spread but not aggregated convert to %
    es_pred = effective_spread(X_print[classifier], X_print["TRADE_PRICE"], mid, mode="none")
    eps_pred = np.empty(es_pred.shape)
    np.divide(es_pred, mid, out=eps_pred, where=mid != 0)

    wilcoxon_res  = wilcoxon(eps_pred, eps_true, nan_policy="omit", zero_method="zsplit")

    res = pd.Series(
            {
                "nom_pred": nom_pred * 100,
                "rel_pred": rel_pred * 100,
                "statistic":wilcoxon_res.statistic,
                "pvalue":wilcoxon_res.pvalue,
            }, name=classifier
        )
    results.append(res)

true_eff = pd.Series({"nom_pred":nom_true * 100, "rel_pred": rel_true * 100, "statistic":np.NaN, "pvalue":np.NaN}, name="true_eff")
true_quoted = pd.Series({"nom_pred":nom_quoted * 100, "rel_pred": rel_quoted * 100, "statistic":np.NaN, "pvalue":np.NaN}, name="true_quoted")

results.append(true_eff)
results.append(true_quoted)

results = pd.concat(results, axis=1)

  0%|          | 0/10 [00:00<?, ?it/s]

In [23]:
results.T.style.format("{:.3f}")


,nom_pred,rel_pred,statistic,pvalue
"('gbm', 'gbm(classical)')",7.414,9.186,15449198918565.000,0.000
"('gbm', 'gbm(classical-size)')",4.247,6.983,18996716578607.500,0.000
"('gbm', 'gbm(ml)')",3.741,6.513,19613712891116.500,0.000
"('classical', 'tick(ex)')",1.557,1.073,22593375152463.500,0.000
"('classical', 'quote(ex)')",16.333,16.207,9330692653233.000,0.000
"('classical', 'lr(ex)')",16.333,16.207,9330692653233.000,0.000
"('classical', 'emo(ex)')",4.647,8.443,21566549392574.500,0.000
"('classical', 'clnv(ex)')",11.626,13.284,18544330865464.000,0.000
"('classical', 'quote(best)->quote(ex)')",6.271,9.419,15830405879950.500,0.000
"('classical', 'trade_size(ex)->quote(best)->quote(ex)->depth(best)->depth(ex)->rev_tick(all)')",1.415,4.293,20692020255975.000,0.000


In [24]:
results.style.to_latex(
    f"../reports/Content/{KEY}-eff-spread.tex",
    siunitx=True,
    position_float="centering",
    hrules=True,
    clines="skip-last;data",
    label=f"tab:eff-{KEY}",
    caption=(f"long-eff-{KEY}", f"short-eff-{KEY}"),
    convert_css=True,
)


## Diffs 🔄️

In [21]:
# classical baselines

base = master[
    [
        ("classical", "quote(best)->quote(ex)"),
        (
            "classical",
            "trade_size(ex)->quote(best)->quote(ex)->depth(best)->depth(ex)->rev_tick(all)",
        ),
        (
            "classical",
            "trade_size(ex)->quote(best)->quote(ex)->depth(best)->depth(ex)->rev_tick(all)",
        ),
    ]
]

# my ml models
revised = master[
    [(MODELS[0], f"{MODELS[0]}(classical)"), (MODELS[0], f"{MODELS[0]}(classical-size)"), (MODELS[0], f"{MODELS[0]}(ml)")]
]


In [22]:
def combine_results(revised: pd.DataFrame, base: pd.DataFrame) -> pd.DataFrame:
    """
    Generate print layout like in Grauer et al.

    https://tex.stackexchange.com/questions/430283/table-with-numbers-in-parentheses-in-siunitx/430290#430290

    # see p. https://texdoc.org/serve/siunitx/0
    """
    # first, second layer of colum index
    c_1 = revised.columns.get_level_values(1)
    c_2 = ["nom"]
    midx = pd.MultiIndex.from_product([c_1, c_2])

    # copy data from revised add as (column, "nom")
    combo = pd.DataFrame(revised.values, index=revised.index, columns=midx)

    for i, mul_col in enumerate(combo.columns):

        combo[(mul_col[0], "pm")] = (combo[mul_col] - base.iloc[:, i]).round(2)
        combo.sort_index(axis=1, inplace=True)

    return combo


In [23]:
diff = combine_results(revised, base)

diff.style.to_latex(
    f"../reports/Content/diff-{KEY}.tex",
    siunitx=True,
    position_float="centering",
    hrules=True,
    clines="skip-last;data",
    label=f"tab:diff-{KEY}",
    caption=(f"long-diff-{KEY}", f"short-diff-{KEY}"),
    convert_css=True,
)


In [24]:
diff


fttransformer(classical)        \
                                                    nom    pm   
Option Type       C                           61.552726  2.67   
                  P                           62.942856  2.17   
Security Type     Index option                53.741246 -4.06   
                  Others                      67.140345  1.90   
                  Stock option                60.345998  2.76   
Trade Size        (0,1]                       60.574816  2.36   
                  (1,3]                       61.065922  2.89   
                  (3,5]                       61.223988  2.74   
                  (5,11]                      64.294364  2.00   
                  >11                         65.643297  2.22   
Year              2015                        59.149311  3.01   
                  2016                        62.405291  2.51   
                  2017                        62.757030  2.08   
Time to Maturity  <= 1                        62.822214  2.24   
                  (1-2]                       63.277653  2.73   
                  (2-3]                       61.796898  2.08   
                  (3-6]                       60.605037  2.73   
                  (6-12]                      60.658303  2.98   
                  > 12                        54.980419  4.04   
Moneyness         <= 0.7                      64.418661  2.97   
                  (0.7-0.9]                   65.849035  2.22   
                  (0.9-1.1]                   62.596430  2.36   
                  (1.1-1.3]                   53.180114  3.16   
                  > 1.3                       51.737642  3.02   
Location to Quote at mid                      61.401592  5.45   
                  inside                      60.753892  0.55   
                  at quotes                   67.592344  7.62   
                  outside                     60.041602 -6.57   
                  unknown                     77.824153  1.48   
All trades        all                         62.201670  2.44   

                               fttransformer(classical-size)        \
                                                         nom    pm   
Option Type       C                                70.755864  3.76   
                  P                                71.672920  4.23   
Security Type     Index option                     56.597390 -1.93   
                  Others                           75.306743  5.17   
                  Stock option                     69.743643  3.59   
Trade Size        (0,1]                            71.734785  2.95   
                  (1,3]                            72.174763  3.33   
                  (3,5]                            71.912851  2.95   
                  (5,11]                           70.025028  6.73   
                  >11                              69.348202  4.80   
Year              2015                             68.112642  4.76   
                  2016                             71.430327  4.31   
                  2017                             71.646482  2.94   
Time to Maturity  <= 1                             71.513974  4.40   
                  (1-2]                            71.695599  3.58   
                  (2-3]                            70.753307  2.87   
                  (3-6]                            70.267933  2.82   
                  (6-12]                           70.127499  2.66   
                  > 12                             68.230785  3.52   
Moneyness         <= 0.7                           70.345507  6.23   
                  (0.7-0.9]                        72.879601  5.01   
                  (0.9-1.1]                        71.774622  3.62   
                  (1.1-1.3]                        65.846414  4.18   
                  > 1.3                            62.633377  2.66   
Location to Quote at mid                           71.010869  5.11   
                  inside                           67.336010